In [ ]:
%load_ext autoreload
%autoreload 2
from scipy import sparse as sp
from matplotlib import pyplot as plt
import numpy as np
import aklt
from functools import reduce

sigma_z = aklt.Sz.todense()

UP = 1
DOWN = 0

In [ ]:
def get_ground_state_array(L: int) -> np.ndarray:
    ud = np.zeros(4)
    du = np.zeros(4)
    ud[int(f"{UP}{DOWN}", base=2)] = 1
    du[int(f"{DOWN}{UP}", base=2)] = 1
    u = np.zeros(2)
    u[UP] = 1
    singlesite = (ud - du) / np.sqrt(2)
    mat = reduce(sp.kron, [u] + [singlesite]*((L-2)//2) + [u])
    return np.asarray(mat.todense()).ravel()

def get_ground_state_MPS(L: int) -> list[np.ndarray]:
    M_o_u = np.array([2**-0.5, 0]).reshape(1, -1)
    M_o_d = np.array([0, -2**-0.5]).reshape(1, -1)
    M_e_u = np.array([0, 1]).reshape(-1, 1)
    M_e_d = np.array([1, 0]).reshape(-1, 1)
    
    M_o = np.array([M_o_u, M_o_d]).swapaxes(0, 1)
    M_e = np.array([M_e_u, M_e_d])

    edge = np.zeros((1, 2, 1))
    edge[:, UP, :] = 1

    return [edge] + [M_o, M_e] * ((L-2)//2) + [edge]


In [ ]:
# Convince myself that the given MPS format is the same as the product of singlet states
L = 8
Ms = get_ground_state_MPS(L)
# print(*(M.shape for M in Ms))
MSt = aklt.MPS_list_to_tensor(Ms)
psi = get_ground_state_array(L)

# We pick up a minus somewhere ig
assert np.allclose(-MSt.ravel(), psi)
del L, Ms, psi

In [ ]:
Ms = get_ground_state_MPS(12)
# aklt.apply_op_at_site(Ms, sigma_z, 1, copy=False)
sij = aklt.operator_correlation(Ms, sigma_z)

plt.figure()
plt.imshow(sij)
plt.colorbar()
plt.show()

In [ ]:
S_OP = np.zeros((2, 3, 2))
S_OP[UP, 0, UP] = 1
S_OP[UP, 1, DOWN] = 1 / np.sqrt(2)
S_OP[DOWN, 1, UP] = 1 / np.sqrt(2)
S_OP[DOWN, 2, DOWN] = 1

def s1_projector(MPS_L: np.ndarray, MPS_R: np.ndarray) -> np.ndarray:
    singlet = np.tensordot(MPS_L, MPS_R, axes=(-1,0))
    return np.tensordot(S_OP, singlet, axes=((0, -1), (1, 2))).swapaxes(0, 1)

In [ ]:
L = 50
Ms = get_ground_state_MPS(L)
S = [s1_projector(Ms[i], Ms[i + 1]) for i in range(0, L, 2)]

print(f"Before normalisation: {aklt.overlap(S, S) = :.3e}")

# I don't understand why this is correct. Stolen from solution
for i in range(len(S)):
    S[i] *= np.sqrt(4/3)

print(f"After normalisation : {aklt.overlap(S, S) = :.3e}")


In [ ]:
# From https://en.wikipedia.org/wiki/3D_rotation_group#A_note_on_Lie_algebras 
# we get the spin operator for S=1:
S1z = np.diag([1, 0, -1])

In [ ]:
sij = aklt.operator_correlation(S, S1z)

plt.figure()
plt.imshow(sij)
plt.colorbar()
plt.show()

In [ ]:

# Ugly plot code, please ignore
hist1 = {}
for i in range(len(S)):
    for j in range(len(S)):
        d = abs(i - j)
        if hist1.get(d) is None:
            hist1[d] = []
        hist1[d].append(sij[i, j])

hist12 = {}
sij = aklt.operator_correlation(Ms, sigma_z)
for i in range(L//2):
    for j in range(L//2):
        d = abs(i - j)
        if hist12.get(d) is None:
            hist12[d] = []
        hist12[d].append(sij[i, j])

plt.figure()
plt.title("$\\left<S^z_i S^z_j\\right>$ for the AKLT ground state")
plt.scatter(hist12.keys(), [max(np.abs(vals)) for vals in hist12.values()], label="spin-1/2")
plt.scatter(hist1.keys(), [max(np.abs(vals)) for vals in hist1.values()], label="spin-1")
plt.legend()
plt.xlabel("$|i - j|$")

plt.show()